In [1]:
import os
import re
os.chdir('../')

In [121]:
from src.general_utils import util

In [2]:
import json
import requests
import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

/Users/manish.sahu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
cities = [
    'bengaluru',
    # 'national-capital-region-ncr',
]

In [4]:
# https://in.bookmyshow.com/explore/movies-bengaluru?languages=hindi
# language = '?languages=hindi'
# path_base_url = 'https://in.bookmyshow.com/explore/movies-'

path_base_url = 'https://paytm.com/movies/'

delay = 300 # Delays the script execution
headers = {
'Referer': 'https://in.bookmyshow.com/explore/movies-bengaluru?languages=hindi',
'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': "Linux",
'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}

In [5]:
for city in cities:
    path_url =  path_base_url + city
    print(f'path_url: {path_url}')

path_url: https://paytm.com/movies/bengaluru


In [6]:
response = requests.get(path_url, headers = headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
class_one = 'MovieCityLandingDesktop_outercontainer__WJTmM'

In [8]:
result = soup.findAll('div')
print(f'result: {len(result)}')

result: 605


In [9]:
movie_list = soup.findAll('div', class_=re.compile("moviesList"))
print(f'result: {len(movie_list)}')

result: 1


In [10]:
running_movie_list = movie_list[0].findAll('div', class_=re.compile("RunningMovie_details"))
print(f'result: {len(running_movie_list)}')

result: 28


In [12]:
movie_data = []

for index in tqdm(range(len(running_movie_list))):
    temp_running_movie = running_movie_list[index]

    temp_data = temp_running_movie.find('a', href=True)
    temp_movie_data = {
        'name': temp_data.text,
        'url': temp_data.get('href')
    }
    
    movie_data.append(temp_movie_data)



100%|██████████| 28/28 [00:00<00:00, 29441.09it/s]


In [13]:
running_movie_list[1]

<div class="MobileRunningMovie_details__oEcin"><div class="MobileRunningMovie_rowMovie__Tr9_Z"><div class="MobileRunningMovie_detMovie__YBAdj"><a class="MobileRunningMovie_lineClamp__akkn9 MobileRunningMovie_clampTwo__eTC0p" href="/movies/kushi-movie-detail-154164">Kushi</a><span class="MobileRunningMovie_lineClamp__akkn9 MobileRunningMovie_clampOne__0_q0w"><span>U/A</span>Telugu</span></div><div class="MobileRunningMovie_butBook__gc8PY"><div><button class="Button_btn__NHo2w Button_is-secondary__J8lii Button_is-medium__qKDWu">Book Now</button></div></div></div><div class="MobileRunningMovie_tag__x1KWx"><span>2D</span></div></div>

In [138]:
path_movie_url = 'https://in.bookmyshow.com/buytickets/pathaan-bengaluru/movie-bang-ET00323848-MT/20230203'
response = requests.get(path_movie_url, headers = headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
  

In [112]:
class GetMovieDetails:
    def __init__(self, path_movie_url):
        self.path_movie_url = path_movie_url    

    def _get_header(self):
        headers = {
                'Referer': 'https://in.bookmyshow.com/explore/movies-bengaluru?languages=hindi',
                'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': "Linux",
                'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                }
        return headers
        
    def _get_soup(self):
        headers = self._get_header()

        response = requests.get(self.path_movie_url, headers = headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    
    def _get_chrome_driver(self):
        # driver = webdriver.Chrome(ChromeDriverManager().install())
        driver = webdriver.Chrome()
        driver.get(self.path_movie_url)
        return driver

    
    def convert_to_json(self, data_categories:str):
        return json.loads(data_categories)

    def get_price(self, data_json):
        return data_json.get('price')

    def get_desc(self, data_json):
        return data_json.get('desc')

    def get_availability_class(self, data_json):
        return data_json.get('availabilityClass')

    def get_availStatus(self, data_json):
        return data_json.get('availStatus')

    def get_processed_df(self):
        driver = self._get_chrome_driver()
        
        print(driver)
        theatre_element_list = driver.find_elements(By.CLASS_NAME, 'list')
        print(f'Number of theatres : {len(theatre_element_list)}')

        data = []

        for i, a in tqdm(enumerate(theatre_element_list)):
            try:
                titles = a.find_element(By.CLASS_NAME, '__title')
                listing_info = a.find_elements(By.CLASS_NAME, 'listing-info')
                showtimes = a.find_elements(By.CLASS_NAME, 'showtime-pill')

                for s in showtimes:

                    data_categories = self.convert_to_json(s.get_attribute('data-categories'))
                    timings = s.get_attribute('data-date-time')

                    temp_data = {
                        'title' : titles.text,
                        'timings': timings,
                        'latitude': a.get_attribute('data-lat'),
                        'longtitude' : a.get_attribute('data-lng'),

                        'price': self.get_price(data_categories),
                        'description': self.get_desc(data_categories),

                        'availability_class': self.get_availability_class(data_categories),
                        'avail_status': self.get_availStatus(data_categories)

                    }
                    
                    data.append(temp_data)

                # b.find_element('showtime-pill')
            except:
                print(i)

        
        data_df = pd.DataFrame(data)

        return data_df
        

In [32]:
path_url = 'https://in.bookmyshow.com/buytickets/pathaan-bengaluru/movie-bang-ET00323848-MT/20230204'
path_url = 'https://in.bookmyshow.com/buytickets/jawan-bengaluru/movie-bang-ET00330424-MT/20230916'
# https://in.bookmyshow.com/bengaluru/movies/jawan/ET00330424'
bmys = GetMovieDetails(path_movie_url=path_url)

In [ ]:
data_df = bmys.get_processed_df()
data_df.head()

In [35]:
data_df

,title,timings,latitude,longtitude,price,description,availability_class,avail_status
0,"Alankar Theater Hoskote Dolby Atmos, 2K Laser\...",04:00 PM,13.0784,77.7867,100.00,Gold,_available,3
1,"Alankar Theater Hoskote Dolby Atmos, 2K Laser\...",07:00 PM,13.0784,77.7867,100.00,Gold,_available,3
2,Anjan Digital 4K A/C Cinema: Magadi Road\nINFO,07:05 PM,12.9754,77.5589,100.00,SECOND CLASS,_available,3
3,Anjan Digital 4K A/C Cinema: Magadi Road\nINFO,10:25 PM,12.9754,77.5589,100.00,SECOND CLASS,_available,3
4,"Ashoka Theatre: Chikkabanavara, Laser Digital\...",04:00 PM,13.0807,77.5011,100.00,First Class,_available,3
...,...,...,...,...,...,...,...,...
510,Victory Cinema Barco-4K RGB-Laser: Kamakshipal...,03:30 PM,12.983,77.5302,195.00,PREMIUM,_available,3
511,Victory Cinema Barco-4K RGB-Laser: Kamakshipal...,07:00 PM,12.983,77.5302,195.00,PREMIUM,_available,3
512,Victory Cinema Barco-4K RGB-Laser: Kamakshipal...,10:30 PM,12.983,77.5302,195.00,PREMIUM,_available,3
513,Vinayaka Cinemas 4K Dolby 11.5 A/C 3D: Harinag...,04:00 PM,12.8751,77.5699,120.00,DIAMOMD,_available,3


In [126]:
class GetMoviesList:
    def __init__(self, path_movie_list_url):
        self.path_movie_list_url = path_movie_list_url    
        self.movie_ahref_string = 'https://in.bookmyshow.com/bengaluru/movies'

    def _get_header(self):
        headers = {
                'Referer': 'https://in.bookmyshow.com/explore/movies-bengaluru?languages=hindi',
                'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': "Linux",
                'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                }
        return headers

    def _get_chrome_driver(self):
        # driver = webdriver.Chrome(ChromeDriverManager().install())
        driver = webdriver.Chrome()
        driver.get(self.path_movie_list_url)
        
        # Scroll to buttom slowly to load all movies
        driver.execute_async_script(
            """
        count = 400;
        let callback = arguments[arguments.length - 1];
        t = setTimeout(function scrolldown(){
            console.log(count, t);
            window.scrollTo(0, count);
            if(count < (document.body.scrollHeight || document.documentElement.scrollHeight)){
              count+= 400;
              t = setTimeout(scrolldown, 1000);
            }else{
              callback((document.body.scrollHeight || document.documentElement.scrollHeight));
            }
        }, 1000);"""
        )
        return driver
    
    def parse_drive_to_soup(self, driver):
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        return soup
    
    def get_all_current_movie_list(self, soup):
        
        movie_soup_element_dict = {}
        
        movie_tags_list = [a for a in soup.find_all('a', href=True) if self.movie_ahref_string in a['href']]
        
        # Extract movie names from the URLs
        movie_names = []
        for link in movie_tags_list:
            # Extract the movie name from the URL
            movie_name_match = re.search(r'.*/movies/(.*)/ET.*', link['href'])
            if movie_name_match:
                movie_name = movie_name_match.group(1)
                movie_names.append(movie_name)

                movie_soup_element_dict[movie_name] = {
                    # 'element': link,
                    'url': link['href'],
                    'name': movie_name,
                }
            
        # # Print the movie names
        # for movie_name in movie_names:
        #     print("Movie Name:", movie_name)

        
        print(f'Number of movies running: {len(movie_names)}')
        return movie_soup_element_dict
        
    def generate(self):
        driver = self._get_chrome_driver()
        soup = self.parse_drive_to_soup(driver)
        movie_soup_element_dict  = self.get_all_current_movie_list(soup)
        
        return movie_soup_element_dict

In [118]:
path_movie_list_url = 'https://in.bookmyshow.com/explore/movies-bengaluru'
# driver = webdriver.Chrome()
# driver.get(path_movie_list)

In [119]:
movie_list_instance = GetMoviesList(path_movie_list_url=path_movie_list_url)
movie_soup_element_dict = movie_list_instance.generate()

Number of movies running: 27


In [131]:
movie_soup_element_dict = {i: {'url': movie_soup_element_dict[i]['url'], 'name': movie_soup_element_dict[i]['name'] } for i in movie_soup_element_dict}

In [132]:
util.save_json(movie_soup_element_dict, path_output='data/scrapped_16_09_2023.json')

In [204]:
# # for holding the resultant list
# element_list = []
  
# for page in range(1, 3, 1):
    
#     page_url = "https://webscraper.io/test-sites/e-commerce/static/computers/laptops?page=" + str(page)
#     driver = webdriver.Chrome(ChromeDriverManager().install())
#     driver.get(page_url)
#     title = driver.find_elements_by_class_name("title")
#     price = driver.find_elements_by_class_name("price")
#     description = driver.find_elements_by_class_name("description")
#     rating = driver.find_elements_by_class_name("ratings")
  
#     for i in range(len(title)):
#         element_list.append([title[i].text, price[i].text, description[i].text, rating[i].text])
  
# print(element_list)
  
# #closing the driver
# driver.close()